In [8]:
bugs_id2name = {1:'purple', 2:'green',3:'yellow', 4:'red'}
bugs_name2id = {name:id for id, name in bugs_id2name.items()}

# encode the bugs
PURPLE = 1 # purple bug
GREEN  = 2 # green and orange bug
YELLOW = 3 # yellow bug
RED    = 4 # lady bug

TOP    =  1 # top half of a bug
BOTTOM = -1 # bottom half of a bug

# tiles description from the image - considered 0 rotation
tiles= {
    1: {"top": TOP*PURPLE,    "bottom": BOTTOM*GREEN,   "left": TOP*YELLOW,   "right": BOTTOM*RED},
    2: {"top": TOP*YELLOW,    "bottom": BOTTOM*GREEN,   "left": TOP*RED,      "right": BOTTOM*YELLOW},
    3: {"top": TOP*GREEN,     "bottom": BOTTOM*PURPLE,  "left": BOTTOM*YELLOW,"right": TOP*RED},
    4: {"top": BOTTOM*GREEN,  "bottom": TOP*PURPLE,     "left": TOP*RED,      "right": TOP*YELLOW},
    5: {"top": TOP*RED,       "bottom": TOP*GREEN,      "left": TOP*PURPLE,   "right": BOTTOM*RED},
    6: {"top": TOP*GREEN,     "bottom": BOTTOM*PURPLE,  "left": TOP*RED,      "right": TOP*YELLOW},
    7: {"top": BOTTOM*PURPLE, "bottom": BOTTOM*RED,     "left": BOTTOM*GREEN, "right": BOTTOM*YELLOW},
    8: {"top": TOP*PURPLE,    "bottom": BOTTOM*GREEN,   "left": BOTTOM*RED,   "right": TOP*YELLOW},
    9: {"top": TOP*PURPLE,    "bottom": BOTTOM*PURPLE,  "left": TOP*YELLOW,   "right": BOTTOM*GREEN},
}

# example solution (from the image): (x_coord, y_coord): (tile_id, rotation))
solution = {(0,0): (5,0), (0,1): (2,0), (0,2):(9,0),
            (1,0): (4,0), (1,1): (3,0), (1,2): (8,0),
            (2,0): (7,0), (2,1): (1,0), (2,2): (6,0)}

# precondition n = 0,1,2,3
# shifts items in a list by n positions
def shift_list(lst, n):
  assert(n >= 0 and n <= 3)
  return lst[-n:] + lst[:-n] # + lst[:n]

# rotates the sides of a tile
# rotation = 0 (no rotation), 1 = 90 clockwise, 2 = 180 degrees clockwise, 3 = 270 degrees clockwise
def rotate_side(side_name, rotation):
  assert(rotation >= 0 and rotation <= 3)
  order_sides = ["top", "right", "bottom", "left"]
  ind_side_name = order_sides.index(side_name)
  rotate_sides = shift_list(order_sides,rotation)
  return rotate_sides[ind_side_name]

# tests okay
def get_side(tile_id, rotation, side_name):# get top of tile, rotation, get bottom of neighbor tile, rotation
  tile0 = tiles[tile_id]
  rotate_side_name = rotate_side(side_name, rotation)
  return tile0[rotate_side_name]

# matching constraints
neighbors = {(0,0): ["right", "bottom"],
             (0,1): ["right", "bottom","left"],
             (0,2): ["bottom", "left"],
             (1,0): ["top", "right", "bottom"],
             (1,1): ["top", "right", "bottom", "left"],
             (1,2): ["top", "bottom", "left"],
             (2,0): ["top", "right"],
             (2,1): ["left", "top", "right"],
             (2,2): ["left", "top"]}

def check_solution(solution): # checks if full solution is valid
  for pos in solution: # for each position assigned in solution
    x,y = pos[0], pos[1]
    tile_id, rotation = solution[pos]
    check_new_tile(solution, pos, solution[pos])
  return True

# check if a new tile matches the other tiles placed
def check_new_tile(solution, new_pos, new_val):
  x, y = new_pos[0], new_pos[1]
  tile_id, rotation = new_val[0], new_val[1]

  # identify the locations you need to check
  for side in neighbors[new_pos]:

    side_tile, side_neighbor = None, None

    if side == "top" and (x-1,y) in solution:      # check pos x-1,y
      neighbor_id, neighbor_rotation = solution[(x-1,y)]
      # get top of tile, rotation, get bottom of neighbor tile, rotation
      side_tile     = get_side(tile_id, rotation, "top")
      side_neighbor = get_side(neighbor_id, neighbor_rotation, "bottom")

    elif side == "bottom" and (x+1,y) in solution: # check pos x+1,y
      neighbor_id, neighbor_rotation = solution[(x+1,y)]
      side_tile     = get_side(tile_id, rotation, "bottom")
      side_neighbor = get_side(neighbor_id, neighbor_rotation, "top")

    elif side == "left" and (x,y-1) in solution:   # check pos x, y-1
      neighbor_id, neighbor_rotation = solution[(x,y-1)]
      side_tile     = get_side(tile_id, rotation, "left")
      side_neighbor = get_side(neighbor_id, neighbor_rotation, "right")

    elif side == "right" and (x,y+1) in solution:  # check pos x, y+1
      neighbor_id, neighbor_rotation = solution[(x,y+1)]
      side_tile     = get_side(tile_id, rotation, "right")
      side_neighbor = get_side(neighbor_id, neighbor_rotation, "left")

      #print(neighbor_id, tiles[neighbor_id], rotation)
      #print(side, side_tile, side_neighbor)

    if side_tile != None and side_neighbor != None:
      if side_tile + side_neighbor != 0:
        return False
  return True


valid_values = []
for tile_id in tiles:
  for rotation in range(4):
    value = (tile_id, rotation)
    valid_values.append(value)

print(len(valid_values))


valid_domain = {key:valid_values for key in solution}
print(valid_domain[(0,0)])

#valid_domain[(0,0)] = (5,0)
print(valid_domain[(0,0)])



def select_unassigned_variable(assigned_var, valid_domains):
  for var in valid_domains:
    if var not in assigned_var:
      return var
  return None

def order_domain_values(var, assigned_var, valid_domains):
  return valid_domains[var]

def is_consistent(var, value, assigned_var):
  return check_new_tile(assigned_var, var, value)


def backtracking_search(valid_domains, assigned_var={}):
    return recursive_backtracking(valid_domains, assigned_var)

def recursive_backtracking(assigned_var, valid_domains):
  if len(assigned_var) == len(valid_domains):
    return assigned_var

  var = select_unassigned_variable(assigned_var, valid_domains)
  for value in order_domain_values(var, assigned_var, valid_domains):
    if is_consistent(var, value, assigned_var):
      assigned_var[var] = value
      result = recursive_backtracking(assigned_var, valid_domains)
      if result != None:
        return result
      del assigned_var[var]
  return None
'''
def forward_tracking_search(valid_domains):
    return recursive_forward_tracking({}, valid_domains)

def recursive_forward_tracking(assigned_var, valid_domains):
    if len(assigned_var) == len(valid_domains):
        return assigned_var

    var2 = min_remaining_values(valid_domains, assigned_var)

    for value in order_domain_values(var2, assigned_var, valid_domains):
        if is_consistent(var2, value, assigned_var):
            assigned_var[var2] = value
            prune = forward_checking(var2, value, valid_domains)


            if prune is not None:
                result = recursive_forward_tracking(assigned_var, valid_domains)
                if result is not None:
                    return result

            restore_pruned_values(var2, value, valid_domains)
            del assigned_var[var2]

    return None

def forward_checking(var, value, valid_domains):
    pruned_values = {}

    for neighbor in neighbors[var]:
        if neighbor not in valid_domains:
            continue

        new_domain = []
        for neighbor_value in valid_domains[neighbor]:
            if is_consistent(neighbor, neighbor_value, {var: value}):
                new_domain.append(neighbor_value)

        if not new_domain:
            return None

        pruned_values[neighbor] = valid_domains[neighbor]
        valid_domains[neighbor] = new_domain

    return pruned_values

def restore_pruned_values(var, value, valid_domains):
    if var in valid_domains:
        valid_domains[var] = value

def min_remaining_values(valid_domains, assigned_var):
    unassigned = [v for v in valid_domains if v not in assigned_var]
    return min(unassigned, key=lambda v: len(valid_domains[v]))


'''


36
[(1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3), (4, 0), (4, 1), (4, 2), (4, 3), (5, 0), (5, 1), (5, 2), (5, 3), (6, 0), (6, 1), (6, 2), (6, 3), (7, 0), (7, 1), (7, 2), (7, 3), (8, 0), (8, 1), (8, 2), (8, 3), (9, 0), (9, 1), (9, 2), (9, 3)]
[(1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3), (4, 0), (4, 1), (4, 2), (4, 3), (5, 0), (5, 1), (5, 2), (5, 3), (6, 0), (6, 1), (6, 2), (6, 3), (7, 0), (7, 1), (7, 2), (7, 3), (8, 0), (8, 1), (8, 2), (8, 3), (9, 0), (9, 1), (9, 2), (9, 3)]


'\n\n\n\nfunction FORWARD-TRACKING-SEARCH(csp) returns a solution, or failure\n    return RECURSIVE-FORWARD-TRACKING({}, csp)\n\nfunction RECURSIVE-FORWARD-TRACKING(assignment, csp) returns a solution, or failure\n    if assignment is complete then return assignment\n    var ← MIN-REMAINING-VALUES(Variables[csp], assignment, csp)\n    for each value in ORDER-DOMAIN-VALUES(var, assignment, csp) do\n        if value is consistent with assignment according to Constraints[csp] then\n            add { var = value } to assignment\n            prune ← FORWARD-CHECKING(var, value, csp)\n            if prune ≠ failure then\n                result ← RECURSIVE-FORWARD-TRACKING(assignment, csp)\n                if result ≠ failure then return result\n            restore pruned values\n            remove { var = value } from assignment\n    return failure\n\nfunction FORWARD-CHECKING(var, value, csp) returns success or failure\n    for each neighbor in Neighbors[var, csp] do\n        remove inconsi